In [1]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
import lsst.sims.maf.db as db

In [3]:
opsdb = db.OpsimDatabase('db/minion_1016_sqlite.db')

In [4]:
constraint = '(fieldid=744 or fieldid=1427 or fieldid=2412 or fieldid=2786) and propid=56'
colnames = ['expMJD', 'fieldID', 'filter']
data = opsdb.fetchMetricData(colnames, constraint)

In [5]:
constraint = ''
colnames = ['expMJD']
alltimes = opsdb.fetchMetricData(colnames, constraint)

In [6]:
fields = np.unique(data['fieldID'])
print '4 DD fieldids', fields

4 DD fieldids [ 744 1427 2412 2786]


In [7]:
# Remember times are in days from opsim.
timereq = {}
for field in fields:
    times = data['expMJD'][np.where(data['fieldID'] == field)]
    gaps = np.concatenate([[0], (np.where(np.diff(times) > 2*120./60./24.)[0])+1])
    tstart_seq = times[gaps]
    tend_seq = times[np.roll(gaps, -1) - 1]
    timereq[field] = (tend_seq - tstart_seq).sum()*24.0

In [8]:
print 'time per field'
print timereq

time per field
{744: 266.47123199771158, 2786: 220.99619999999413, 1427: 266.44963200209895, 2412: 237.10456800047541}


In [9]:
time = 0
for field in fields:
    time += timereq[field]

In [10]:
print 'total 4 pre-select DD fields, time required:', time, time/10.0
print 'average time per field', time/4.0

total 4 pre-select DD fields, time required: 991.021632 99.1021632
average time per field 247.755408


In [11]:
timeavail = 0
allgaps = np.concatenate([[0], (np.where(np.diff(alltimes['expMJD']) > 2./24.)[0])+1])
tup = alltimes['expMJD'][allgaps]
tdown = alltimes['expMJD'][np.roll(allgaps, -1)-1]

In [12]:
uptime = (tdown - tup).sum()*24.0
print 'total available time', uptime

total available time 27779.077944


In [13]:
print 'Nvisits', len(data), 'Total visits', len(alltimes)
print '%.1f percent of total visits' % (float(len(data))/len(alltimes)*100)

Nvisits 88874 Total visits 2447931
3.6 percent of total visits


In [14]:
print 'Time used for 4 dd fields', time/uptime*100.

Time used for 4 dd fields 3.56751089434


In [15]:
totseq = len(tstart_seq)
shortseq = len(np.where((tend_seq - tstart_seq)*24.*60 < 15)[0])
longseq = len(np.where((tend_seq - tstart_seq)*24.*60 > 50)[0])
print 'total number of sequences', totseq
print 'number of long sequences', longseq
print 'number of short sequences', shortseq
print float(longseq)/shortseq, longseq+shortseq

total number of sequences 367
number of long sequences 168
number of short sequences 199
0.844221105528 367


In [16]:
print 'filters used in short sequences'
seqfilter = data['filter'][gaps][np.where((tend_seq - tstart_seq)*24.*60 > 50)]
nonu = seqfilter[np.where(seqfilter!='u')]
print seqfilter
print 'Number of non-u band short sequences:', len(nonu)

filters used in short sequences
['u' 'u' 'r' 'r' 'r' 'i' 'i' 'r' 'r' 'r' 'i' 'i' 'r' 'r' 'y' 'z' 'u' 'r'
 'r' 'r' 'r' 'y' 'y' 'y' 'i' 'u' 'u' 'i' 'i' 'z' 'z' 'z' 'z' 'z' 'z' 'u'
 'y' 'y' 'z' 'z' 'z' 'z' 'z' 'z' 'z' 'r' 'r' 'r' 'r' 'u' 'u' 'u' 'u' 'g'
 'g' 'g' 'g' 'g' 'y' 'y' 'y' 'y' 'i' 'u' 'u' 'u' 'g' 'g' 'g' 'y' 'y' 'y'
 'y' 'y' 'z' 'z' 'u' 'u' 'z' 'z' 'i' 'r' 'r' 'r' 'u' 'z' 'z' 'z' 'r' 'z'
 'u' 'r' 'r' 'u' 'g' 'g' 'z' 'z' 'u' 'i' 'i' 'i' 'i' 'y' 'y' 'u' 'u' 'r'
 'r' 'r' 'r' 'r' 'z' 'z' 'z' 'y' 'y' 'y' 'u' 'y' 'y' 'y' 'y' 'y' 'y' 'u'
 'u' 'u' 'i' 'i' 'y' 'y' 'y' 'y' 'r' 'u' 'u' 'u' 'y' 'y' 'y' 'y' 'y' 'z'
 'z' 'y' 'y' 'g' 'g' 'z' 'z' 'z' 'z' 'i' 'r' 'r' 'r' 'z' 'z' 'z' 'z' 'u'
 'u' 'u' 'u' 'y' 'y' 'r']
Number of non-u band short sequences: 136
